<a href="https://colab.research.google.com/github/Mkr-mayank/CNN/blob/main/Cifar100_Using_Efficientnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
dataset_path='/content/drive/MyDrive/Dataset3/Skin Cancer'
IMG_SIZE=224
BATCH_SIZE=32

In [9]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='sparse',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)

Found 5382 images belonging to 1 classes.
Found 1345 images belonging to 1 classes.


In [10]:
base_model = EfficientNetB0(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.25),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.25),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.25),
    layers.Dense(100, activation='softmax')  # Adjust this based on your dataset's number of classes
])

In [11]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_gen, validation_data=val_gen, epochs=5)

Epoch 1/5
169/169 [==============================] - 156s 868ms/step - loss: 3.7139 - accuracy: 0.4208 - val_loss: 1.0475 - val_accuracy: 1.0000


In [ ]:
loss, accuracy = model.evaluate(val_gen)
print(f'Validation accuracy: {accuracy*100:.2f}%')